# ENV

In [ ]:
!apt install python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!update-alternatives --set python3 /usr/bin/python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py
!pip --version
!python3 -m pip install setuptools==59.5.0
!pip install virtualenv
!python3 -m virtualenv venv


In [ ]:
# !source /content/venv/bin/activate; pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
!source venv/bin/activate; pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!source /content/venv/bin/activate; pip install -e .

In [ ]:
%cd ..
!git clone https://github.com/ayanban011/SwinDocSegmenter.git
%cd SwinDocSegmenter


In [ ]:
%cd /content/SwinDocSegmenter/

In [ ]:
!source /content/venv/bin/activate; pip install -r requirements.txt


In [ ]:
%cd maskdino/modeling/pixel_decoder/ops
!source /content/venv/bin/activate; sh make.sh

# DATA

In [ ]:
!pip install gdown

In [ ]:
!gdown -O publaynet.zip --id 1qvvj4QYWlXZHYAgahEYUfH2UUmbX6u_q
!unzip publaynet.zip

In [21]:
!export DETECTRON2_DATASETS=/content/content

In [ ]:
!source /content/venv/bin/activate;pip install git+https://github.com/cocodataset/panopticapi.git

In [ ]:
!source /content/venv/bin/activate; python /content/SwinDocSegmenter/datasets/prepare_coco_semantic_annos_from_panoptic_annos.py

# TRAINING

In [17]:
%cd /content/SwinDocSegmenter

/content/SwinDocSegmenter


In [18]:
!cp /content/SwinDocSegmenter/configs/coco/instance-segmentation/Base-COCO-InstanceSegmentation.yaml /content/base.yaml

In [19]:
#@title config
cfg = """
_BASE_: /content/base.yaml
MODEL:
  META_ARCHITECTURE: "MaskDINO"
  SEM_SEG_HEAD:
    NAME: "MaskDINOHead"
    IGNORE_VALUE: 255
    NUM_CLASSES: 11
    LOSS_WEIGHT: 1.0
    CONVS_DIM: 256
    MASK_DIM: 256
    NORM: "GN"
    # pixel decoder
    PIXEL_DECODER_NAME: "MaskDINOEncoder"
    DIM_FEEDFORWARD: 2048
    NUM_FEATURE_LEVELS: 3
    TOTAL_NUM_FEATURE_LEVELS: 4
    IN_FEATURES: ["res2", "res3", "res4", "res5"]
    DEFORMABLE_TRANSFORMER_ENCODER_IN_FEATURES: ["res2","res3", "res4", "res5"]
    COMMON_STRIDE: 4
    TRANSFORMER_ENC_LAYERS: 6
    FEATURE_ORDER: "low2high"
  MaskDINO:
    TRANSFORMER_DECODER_NAME: "MaskDINODecoder"
    DEEP_SUPERVISION: True
    NO_OBJECT_WEIGHT: 0.1
    CLASS_WEIGHT: 4.0
    MASK_WEIGHT: 5.0
    DICE_WEIGHT: 5.0
    BOX_WEIGHT: 5.0
    GIOU_WEIGHT: 2.0
    HIDDEN_DIM: 256
    NUM_OBJECT_QUERIES: 300
    NHEADS: 8
    DROPOUT: 0.0
    DIM_FEEDFORWARD: 1024
    ENC_LAYERS: 0
    PRE_NORM: False
    ENFORCE_INPUT_PROJ: False
    SIZE_DIVISIBILITY: 32
    DEC_LAYERS: 9  # 9+1, 9 decoder layers, add one for the loss on learnable query
    TRAIN_NUM_POINTS: 12544
    OVERSAMPLE_RATIO: 3.0
    IMPORTANCE_SAMPLE_RATIO: 0.75
    EVAL_FLAG: 1
    INITIAL_PRED: True
    TWO_STAGE: True
    DN: "seg"
    DN_NUM: 100
    INITIALIZE_BOX_TYPE: 'no'
    TEST:
      SEMANTIC_ON: False
      INSTANCE_ON: True
      PANOPTIC_ON: False
      OVERLAP_THRESHOLD: 0.8
      OBJECT_MASK_THRESHOLD: 0.25
DATASETS:
  TRAIN: ("doc_train",)
  TEST: ("doc_val",)
SOLVER:
  AMP:
    ENABLED: True
DATALOADER:
  NUM_WORKERS: 2
TEST:
  EVAL_PERIOD: 5000
#  EVAL_FLAG: 1
OUTPUT_DIR: "/content/output_doclay_comp_Res"
"""
with open("/content/cfg.yaml", "w") as f:
    f.write(cfg)


In [ ]:
!gdown -O /content/weight.pth --id 1DCxG2MCza_z-yB3bLcaVvVR4Jik00Ecq

In [ ]:
!source /content/venv/bin/activate; python train_net.py --num-gpus 1 \
    --config-file /content/cfg.yaml \
    SOLVER.IMS_PER_BATCH 4 SOLVER.BASE_LR 0.000025 MODEL.NUM_CLASSES 8 \
    SOLVER.STEPS: 3000 4000  SOLVER.MAX_ITER: 16000 \
    DATASETS.TRAIN "/content/content/coco/train" \
    DATASETS.TEST "/content/content/coco/val" \
    MODEL.WEIGHTS "/content/weight.pth"